In [13]:
import json
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import os
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
import requests
from datetime import datetime, timedelta
import classes_func
from classes_func import *

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [14]:
#Creating a search instance with Search('City',start_day,end_day)
Barcelona = Search('Barcelona','2024-02-09','2024-02-13')

In [15]:
#Remove Cookies
#Find destination button
Barcelona.browser.find_element(by='xpath',value= '//button[@id="onetrust-reject-all-handler"]').click()

In [16]:
#Remove Google Sign in
#Barcelona.browser.find_element(by='xpath',value= '//svg[@class="Bz112c Bz112c-r9oPif"]').click()
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


# Define your wait timeout (adjust as needed)
wait_timeout = 10
wait = WebDriverWait(Barcelona.browser, wait_timeout)

try:
    # Switch to the iframe by index (if needed, adjust the index)
    Barcelona.browser.switch_to.frame(0)  # You may need to adjust the index based on your HTML structure

    # Now you can interact with elements within the iframe
    # Wait for the "Not Now" button to be clickable and click it
    not_now_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='TvD9Pc-Bz112c ZYIfFd-aGxpHf-FnSee' and @aria-label='Cerrar']")))
    not_now_button.click()

except Exception as e:
    print(f"An error occurred while handling the Google Sign-In pop-up: {e}")

finally:
    # Switch back to the default content
    Barcelona.browser.switch_to.default_content()


In [17]:
#Inputing city
Barcelona.input_city()

In [18]:
Barcelona.open_date_box()

In [19]:
#Select Date_start
Barcelona.date_selector(Barcelona.start_day)

In [20]:
#Select Date_end
Barcelona.date_selector(Barcelona.end_day)

In [21]:
#Search for Results
Barcelona.search_results()

In [22]:
#Total number of Result pages
pages = Barcelona.result_pages()
pages

32

In [23]:
#Barcelona.scrape_results(pages)
# Also if it's going to slow, adjust the waiting times in the class_function
Barcelona.scrape_results(30)#Add the number of pages you'll like to scrape

page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17
page 18
page 19
page 20
page 21
page 22
page 23
page 24
page 25
page 26
page 27
page 28
page 29
page 30


In [24]:
Barcelona.df.shape

(618, 8)

In [25]:
Barcelona.scrape_descriptions()

Processing Links:   8%|▊         | 49/618 [00:31<06:36,  1.44it/s]

Scraped 50 links


Processing Links:  16%|█▋        | 101/618 [01:02<04:52,  1.77it/s]

Scraped 100 links


Processing Links:  24%|██▍       | 150/618 [01:33<05:32,  1.41it/s]

...

Scraped 550 links


Processing Links:  97%|█████████▋| 600/618 [05:14<00:07,  2.46it/s]

Scraped 600 links


Processing Links: 100%|██████████| 618/618 [05:21<00:00,  1.92it/s]

Scraped 618 links


In [26]:
Barcelona.df.head()

,Hotels,Distance,District,Price,Rating,Stars,Comments,Link,descriptions
0,"SANT ANTONI BREAK BCN, Sustanaible Tourism","a 0,9 km del centro","Eixample, Barcelona",€ 415,"7,5",3 de 5,353 comentarios,https://www.booking.com/hotel/es/pension-sant-...,Apartamento centric es un alojamiento muy bien...
1,Sonder Casa Luz,a 450 m del centro,"Eixample, Barcelona",€ 670,"8,4",4 de 5,2.667 comentarios,https://www.booking.com/hotel/es/casa-luz-barc...,El Sonder Casa Luz se encuentra en Barcelona y...
2,The Social Hub Barcelona Poblenou,"a 3,8 km del centro","Sant Martí, Barcelona",€ 533,"8,8",4 de 5,1.830 comentarios,https://www.booking.com/hotel/es/the-social-hu...,"SANT ANTONI BREAK BCN, Sustanaible Tourism es ..."
3,"Hotel Granados 83, a Member of Design Hotels","a 1,2 km del centro","Eixample, Barcelona",€ 575,"8,5",4 de 5,1.574 comentarios,https://www.booking.com/hotel/es/granados-83.e...,El Wilson Boutique Hotel está situado en la av...
4,Wilson Boutique Hotel,"a 1,8 km del centro","Sarrià-Sant Gervasi, Barcelona",€ 419,"8,5",4 de 5,2.127 comentarios,https://www.booking.com/hotel/es/wilson.es.htm...,"El Granados 83, situado a 250 metros de la est..."


In [27]:
Barcelona.browser.quit()

In [30]:
Barcelona.df.shape

(618, 9)

In [32]:
# Define the file path where you want to save the CSV file
file_path = r'C:/Users/bakwe/Documents/GitHub/Git extra/Bookings/Final_data/Barcelona_09_13.csv'

# Use the to_csv method to save the DataFrame to a CSV file
Barcelona.df.to_csv(file_path, index=False)  # Set index=False if you don't want to save the index column

In [34]:
# Define the file path where the CSV file is located
file_path = r'C:/Users/bakwe/Documents/GitHub/Git extra/Bookings/Final_data/Barcelona_09_13.csv'

# Read the CSV file into a DataFrame
Barcelona_df = pd.read_csv(file_path)
Barcelona_df

,Hotels,Distance,District,Price,Rating,Stars,Comments,Link,descriptions
0,"SANT ANTONI BREAK BCN, Sustanaible Tourism","a 0,9 km del centro","Eixample, Barcelona",€ 415,"7,5",3 de 5,353 comentarios,https://www.booking.com/hotel/es/pension-sant-...,Apartamento centric es un alojamiento muy bien...
1,Sonder Casa Luz,a 450 m del centro,"Eixample, Barcelona",€ 670,"8,4",4 de 5,2.667 comentarios,https://www.booking.com/hotel/es/casa-luz-barc...,El Sonder Casa Luz se encuentra en Barcelona y...
2,The Social Hub Barcelona Poblenou,"a 3,8 km del centro","Sant Martí, Barcelona",€ 533,"8,8",4 de 5,1.830 comentarios,https://www.booking.com/hotel/es/the-social-hu...,"SANT ANTONI BREAK BCN, Sustanaible Tourism es ..."
3,"Hotel Granados 83, a Member of Design Hotels","a 1,2 km del centro","Eixample, Barcelona",€ 575,"8,5",4 de 5,1.574 comentarios,https://www.booking.com/hotel/es/granados-83.e...,El Wilson Boutique Hotel está situado en la av...
4,Wilson Boutique Hotel,"a 1,8 km del centro","Sarrià-Sant Gervasi, Barcelona",€ 419,"8,5",4 de 5,2.127 comentarios,https://www.booking.com/hotel/es/wilson.es.htm...,"El Granados 83, situado a 250 metros de la est..."
...,...,...,...,...,...,...,...,...,...
613,Vacaciones en maresme casa para 7 personas,"a 42,4 km del centro","Ciutat Vella, Barcelona",€ 760,"9,0",4 de 5,23 comentarios,https://www.booking.com/hotel/es/casa-para-7-p...,B42 - MODERNIST APARTMENT FOR LARGE GROUPS IN ...
614,bp1- Piso para Grandes Grupos,"a 0,7 km del centro","Sant Martí, Barcelona",€ 2.598,"7,3",4 de 5,3 comentarios,https://www.booking.com/hotel/es/balmes-apartm...,Bp1- Piso para Grandes Grupos está en pleno co...
615,b42 - MODERNIST APARTMENT FOR LARGE GROUPS IN ...,"a 0,7 km del centro","Sant Martí, Barcelona",€ 2.598,"8,8",3 de 5,12 comentarios,https://www.booking.com/hotel/es/modernist-apa...,"Casa Horta View By MyRentalHost, que tiene jar..."
616,Casa Horta View By MyRentalHost,"a 4,2 km del centro","Ciutat Vella, Barcelona",€ 1.024,"5,0",3 de 5,1 comentario,https://www.booking.com/hotel/es/hortas-house-...,Contemporary villa near Sitges Hills tiene pis...
